In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/syscall

/content/drive/.shortcut-targets-by-id/1e_s52LoUFBat8BLGvGC96XkKahitNbby/syscall


In [7]:
#!python Binary_BERT.py

In [8]:
import pandas as pd
import numpy as np
import re
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torchtext
import torchtext.data.utils as data_utils
import torchtext.vocab as vocab
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [9]:
# Load Data
df = pd.read_csv('ultraclean_final_systemcalls_label.csv').dropna()
df = df.sample(frac=1).reset_index(drop=True)

# Split Data
train_df, test_df = train_test_split(df, random_state=41, train_size=0.8, stratify=df['label'])

# Preprocess Labels
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

In [10]:
df

,apkname,systemcalls,label
0,0074CC33B2B81EFCA8CAFBED4B3C97561044B1A76FA50B...,openat openat openat openat openat openat open...,1
1,00B4BC79BCBF0A062E4066AC893C97560D1CAB4EAC9360...,openat openat openat openat openat openat open...,0
2,00AA7EFCCE69FA707A6ADFA3CE69BBE6C71BEA8E4E567A...,faccessat faccessat faccessat faccessat facces...,0
3,0089A591747499C50DA1B0E128BA2ED7455E5480009BCF...,openat openat openat openat openat openat open...,0
4,igudi.com.huiben,openat openat openat faccessat faccessat opena...,1
...,...,...,...
6844,00837A1AE270E256579C9F5F20EB8A6D7DF904529FB6D2...,faccessat openat newfstatat newfstatat openat ...,1
6845,com.gbxd.mwvj,openat openat openat openat openat openat open...,1
6846,com.ccompanyapps.balonkolamba,openat fstatat64 openat faccessat faccessat fs...,0
6847,com.enttt.errrr,openat openat openat openat openat openat open...,1


In [32]:
pd.set_option('display.max_colwidth', None)

# Display the row
print(df.iloc[1])


apkname                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [11]:
df['label'].value_counts()

label
0    3560
1    3289
Name: count, dtype: int64

In [12]:
# Define Dataset Class
class SystemCallsDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['systemcalls']
        label = self.data.iloc[idx]['label']
        return text, label

In [13]:
# Create Datasets
train_dataset = SystemCallsDataset(train_df)
test_dataset = SystemCallsDataset(test_df)

# Define Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [14]:
# Model Architecture
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

In [15]:
# Function to tokenize text
def tokenize_text(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    return inputs

In [16]:
# Prepare DataLoader
def collate_fn(batch):
    texts, labels = zip(*batch)
    inputs = [tokenize_text(text) for text in texts]
    max_length = max(len(input["input_ids"][0]) for input in inputs)
    padded_input_ids = []
    attention_masks = []
    for input in inputs:
        input_ids = input["input_ids"]
        input_ids_padding = torch.zeros(1, max_length, dtype=torch.long)
        input_ids_padding[:, :input_ids.shape[1]] = input_ids
        padded_input_ids.append(input_ids_padding)
        attention_mask = input["attention_mask"]
        attention_mask_padding = torch.zeros(1, max_length, dtype=torch.long)
        attention_mask_padding[:, :attention_mask.shape[1]] = attention_mask
        attention_masks.append(attention_mask_padding)
    padded_input_ids = torch.cat(padded_input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    return padded_input_ids, attention_masks, labels

In [17]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)

In [18]:
# Define Training Function
def train_model(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        for input_ids, attention_masks, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            outputs = model(input_ids, attention_masks)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct_predictions / total_predictions
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


In [19]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize the classifier model with the base BERT model and number of classes
num_classes = 2
model = BERTClassifier(bert_model, num_classes=num_classes)

# Determine the device to load the model onto (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model weights onto the specified device
model.load_state_dict(torch.load('bert_model.pth', map_location=device))

# Move the model to the appropriate device (GPU if available)
model = model.to(device)

# If you intend to use GPU for inference later, you can check the device again before inference
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))

# Example of using the model for inference
# input_ids and attention_mask should be prepared as per your use case
# inputs = {
#     'input_ids': input_ids.to(device),
#     'attention_mask': attention_mask.to(device)
# }
# outputs = model(**inputs)

KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for input_ids, attention_masks, labels in tqdm(test_loader, desc="Evaluation"):
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            outputs = model(input_ids, attention_masks)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy:.4f}")

    # Print classification report
    print(classification_report(true_labels, predicted_labels))

    return true_labels, predicted_labels, accuracy

# Call the function
true_labels, predicted_labels, accuracy = evaluate_model(model, test_loader)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)

# Classification Report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))

# Weighted F1 Score, Precision, Recall
weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')
weighted_precision = precision_score(true_labels, predicted_labels, average='weighted')
weighted_recall = recall_score(true_labels, predicted_labels, average='weighted')
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")

# Macro F1 Score, Precision, Recall
macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
macro_precision = precision_score(true_labels, predicted_labels, average='macro')
macro_recall = recall_score(true_labels, predicted_labels, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")

# Normalize confusion matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Visualization
plt.figure(figsize=(8, 6))
ax = sns.heatmap(cm_norm, annot=True, cmap='inferno', fmt='.2f', annot_kws={"size": 16})
plt.xlabel('Predicted label', fontsize=16)
plt.ylabel('True label', fontsize=16)

# Define tick marks and positions
tick_marks = np.arange(2)
tick_positions = np.arange(0.5, 2.5, 1)

# Set x-axis tick labels at the center of the cell
ax.set_xticks(tick_positions)
ax.set_xticklabels(['Benign', 'Malware'], fontsize=16, ha='center')

# Set y-axis tick labels at the center of the cell
ax.set_yticks(tick_positions)
ax.set_yticklabels(['Benign', 'Malware'], fontsize=16, va='center')

plt.tight_layout()

# Save figure as EPS
plt.savefig('confusion_matrix.eps', format='eps')

plt.show()




Predicted label for the first data point: Negative


# Explainability

In [ ]:
pip install captum

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# Define the BERTClassifier class
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize your classifier
model = BERTClassifier(bert_model, num_classes=2)

# Load the model state dictionary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load('bert_model.pth', map_location=device))

# Move the model to the appropriate device
model = model.to(device)


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define your custom dataset class
class MyDataset(Dataset):
    def __init__(self, texts, labels, max_length, tokenizer):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        return input_ids, attention_mask, label

# Load Data
df = pd.read_csv('ultraclean_final_systemcalls_label.csv').dropna()
df = df.sample(frac=1).reset_index(drop=True)

# Split Data
train_df, test_df = train_test_split(df, random_state=41, train_size=0.8, stratify=df['label'])

# Preprocess Labels
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

# Example data
texts = test_df['systemcalls'].tolist()
labels = test_df['label'].tolist()
max_length = 128

# Initialize the dataset
test_dataset = MyDataset(texts, labels, max_length, tokenizer)




In [ ]:
# Define collate function
def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    attention_masks = torch.stack([item[1] for item in batch])
    labels = torch.tensor([item[2] for item in batch])
    return input_ids, attention_masks, labels

# Initialize the DataLoader
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


In [74]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
import pandas as pd
from captum.attr import LayerIntegratedGradients, visualization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define your custom dataset class
class MyDataset(Dataset):
    def __init__(self, texts, labels, max_length, tokenizer):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        return input_ids, attention_mask, torch.tensor(label)

# Define collate function
def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    attention_masks = torch.stack([item[1] for item in batch])
    labels = torch.tensor([item[2] for item in batch])
    return input_ids, attention_masks, labels

# Load Data
df = pd.read_csv('ultraclean_final_systemcalls_label.csv').dropna()
df = df.sample(frac=1).reset_index(drop=True)

# Split Data
train_df, test_df = train_test_split(df, random_state=41, train_size=0.8, stratify=df['label'])

# Preprocess Labels
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

# Example data
train_texts = train_df['systemcalls'].tolist()
train_labels = train_df['label'].tolist()
test_texts = test_df['systemcalls'].tolist()
test_labels = test_df['label'].tolist()
max_length = 128

# Initialize the datasets
train_dataset = MyDataset(train_texts, train_labels, max_length, tokenizer)
test_dataset = MyDataset(test_texts, test_labels, max_length, tokenizer)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define the BERTClassifier class
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize your classifier
num_classes = 2  # Change this if you have a different number of classes
model = BERTClassifier(bert_model, num_classes=num_classes)

# Load the model state dictionary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load('bert_model.pth', map_location=device))

# Move the model to the appropriate device
model = model.to(device)

# Evaluation Code (example loop)
model.eval()  # Set the model to evaluation mode

# Select a sample to explain
sample_idx = 0
input_text = test_texts[sample_idx]
label = test_labels[sample_idx]

# Tokenize the input text
encoding = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=max_length,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# Function to perform forward pass
def forward_pass(input_ids, attention_mask):
    outputs = model(input_ids, attention_mask)
    return outputs

# Initialize Layer Integrated Gradients
lig = LayerIntegratedGradients(forward_pass, model.bert.embeddings)

# Get attributions
target = label  # Class index to compute attributions for
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=input_ids * 0,
                                    target=target,
                                    additional_forward_args=(attention_mask,),
                                    return_convergence_delta=True)

# Visualize attributions
attributions = attributions.sum(dim=-1).squeeze(0)
attributions = attributions / torch.norm(attributions)




In [66]:
# Token-level attributions
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
vis_data_records = []
for token, attr in zip(tokens, attributions):
    word_attributions = [attr.item()] * len(token)  # Create a list of attributions for each token
    vis_data_records.append(visualization.VisualizationDataRecord(
        word_attributions=word_attributions,  # Use the list of attributions
        pred_prob=0,  # Provide a default value if not available
        pred_class=str(target),  # Provide a default value if not available
        true_class=str(label),
        attr_class=str(target),
        attr_score=attr.item(),
        raw_input_ids=token,
        convergence_score=delta.item()
    ))

# Render visualization
visualization.visualize_text(vis_data_records)


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,-0.03,[ C L S ]
0,0 (0.00),0,0.06,o p e n
0,0 (0.00),0,0.08,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.08,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.05,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.04,# # a t
0,0 (0.00),0,0.04,o p e n


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,-0.03,[ C L S ]
0,0 (0.00),0,0.06,o p e n
0,0 (0.00),0,0.08,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.08,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.05,# # a t
0,0 (0.00),0,0.04,o p e n
0,0 (0.00),0,0.04,# # a t
0,0 (0.00),0,0.04,o p e n


In [ ]:
import torch
torch.cuda.empty_cache()

In [21]:
pip install transformers-interpret

In [22]:
from transformers import BertForSequenceClassification

# Initialize a BertForSequenceClassification model
model_for_classification = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load your trained weights into the classification model
state_dict = torch.load('bert_model.pth', map_location=torch.device('cpu'))

# Check if the state_dict has keys related to 'classifier' (the default name for the classification head)
if all(k.startswith('classifier') for k in state_dict.keys()):
    # Load the modified state_dict directly
    model_for_classification.load_state_dict(state_dict)
else:
    # Remove the 'fc' layer keys from the state_dict if present
    state_dict = {k: v for k, v in state_dict.items() if 'fc' not in k}
    # Load the modified state_dict
    model_for_classification.load_state_dict(state_dict, strict=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
print(test_texts[sample_idx])

openat openat openat fstatat64 fstatat64 openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat openat 

In [121]:
from torch.nn.functional import softmax
from sklearn.metrics import classification_report
from tqdm import tqdm


# Print explanations in a formatted way
print("Explanations:")
for word, score in word_attributions:  # Assuming word_attributions is a list of tuples (word, score)
    print(f"\t- {word:<20}: {score:.4f}")  # Format word left-aligned up to 20 characters, score with 4 decimals


Explanations:
	- [CLS]               : 0.0000
	- open                : -0.0568
	- ##at                : 0.1260
	- open                : -0.0683
	- ##at                : 0.0480
	- open                : -0.0506
	- ##at                : 0.0402
	- f                   : 0.0053
	- ##sta               : 0.0195
	- ##tat               : 0.0227
	- ##64                : 0.0379
	- f                   : -0.0000
	- ##sta               : 0.0330
	- ##tat               : 0.0559
	- ##64                : 0.0476
	- open                : -0.0591
	- ##at                : 0.0449
	- open                : -0.0404
	- ##at                : 0.0349
	- open                : -0.0405
	- ##at                : 0.0245
	- open                : -0.0426
	- ##at                : 0.0318
	- open                : -0.0272
	- ##at                : -0.0081
	- open                : -0.0376
	- ##at                : 0.0008
	- open                : -0.0452
	- ##at                : -0.0395
	- open                : -0.0196
	- ##at     

In [127]:
pip install bertviz


In [122]:
from torch.nn.functional import softmax
from sklearn.metrics import classification_report
from tqdm import tqdm

def evaluate_model(model, test_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for input_ids, attention_masks, labels in tqdm(test_loader, desc="Evaluation"):
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_masks).logits
            probabilities = softmax(outputs, dim=1)
            _, predicted = torch.max(probabilities, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())

    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy:.4f}")

    # Print classification report
    print(classification_report(true_labels, predicted_labels, digits=4))

    return true_labels, predicted_labels, accuracy

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_for_classification.to(device)

# Call the function
true_labels, predicted_labels, accuracy = evaluate_model(model_for_classification, test_loader, device)


Evaluation:  42%|████▏     | 36/86 [00:27<00:38,  1.31it/s]


KeyboardInterrupt: 

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer
from transformers import BertForSequenceClassification

In [152]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Create explainer
cls_explainer = SequenceClassificationExplainer(model, tokenizer)


In [153]:
# Extract five random sentences based on a pre-selection
random_sentences = df[df.label.isin([0, 1])].sample(n=5, random_state=1234)['systemcalls'].reset_index(drop=True)

# Print each sentence in its full length
for sentence in random_sentences:
    print(sentence)


openat fstatat64 fstatat64 fstatat64 fstatat64 faccessat faccessat mkdirat fchmodat openat faccessat faccessat fstatat64 faccessat openat openat fchmodat fstatat64 fstatat64 openat openat openat openat openat openat openat openat openat openat openat openat openat fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 openat openat fstatat64 fstatat64 openat openat openat openat openat openat openat fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 faccessat faccessat fstatat64 faccessat openat openat openat openat openat openat openat openat openat fstatat64 fstatat64 fstatat64 fstatat64 faccessat faccessat fstatat64 openat openat openat openat openat openat openat openat openat openat openat openat fstatat64 fstatat64 fstatat64 fstatat64 openat unlinkat fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 openat readlinkat fstatat64 fstatat64 fstatat64 openat fstatat64 fstatat64 fstatat64 openat fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 openat 

In [27]:
!pip install bertviz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.6 MB/s eta 0:00:00


In [177]:
from transformers import AutoTokenizer, AutoModel
from bertviz import model_view
import torch

# Cell 3: Load test data
df = pd.read_csv('ultraclean_final_systemcalls_label.csv').dropna()
test_df = df.sample(frac=0.2, random_state=42)  # Adjust fraction as needed

# Cell 4: Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased', output_attentions=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Cell 5: Select a random sample from the test data
random_sample = test_df.sample(1)

# Cell 6: Tokenize the text, remove duplicates, ensure it fits within the maximum sequence length
input_text = random_sample['systemcalls'].iloc[0]
print(f"Input Text: {input_text}")  # Print the original input text

# Remove duplicates from the text (consider using sets or other methods based on your needs)
unique_text = ' '.join(set(input_text.split()))
print(f"Unique Text (after removing duplicates): {unique_text}")

inputs = tokenizer.encode_plus(unique_text, return_tensors='pt', add_special_tokens=True, max_length=512, truncation=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Print some information about the tokenized input for debugging
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Mask shape: {attention_mask.shape}")

# Cell 7: Get the model outputs
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    attention = outputs[-1]  # Retrieve attention from model outputs

# Cell 8: Convert input ids to tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# Cell 9: Visualize attention using BertViz
try:
    model_view(attention, tokens)
except Exception as e:
    print(f"BertViz Visualization Error: {e}")


Input Text: openat openat openat openat fstatat64 fstatat64 fstatat64 fstatat64 openat fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 fstatat64 openat openat 
Unique Text (after removing duplicates): fstatat64 openat
Input IDs shape: torch.Size([1, 8])
Attention Mask shape: torch.Size([1, 8])


<IPython.core.display.Javascript object>